<a href="https://colab.research.google.com/github/unt-iialab/medical-concept-normalization/blob/master/medical-concept-normalization/fine-tuning-experiments/askpatient/cadec_ask_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os

In [2]:
base_dir = 'medical-concept-normalization'

In [3]:
os.mkdir(base_dir)

In [4]:
os.mkdir('medical-concept-normalization/data_collection')

**Install fastai and clone the fastai for text classification**



In [5]:
!curl https://course-v3.fast.ai/setup/colab | bash

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0curl: (6) Could not resolve host: course-v3.fast.ai


In [6]:
!git clone https://github.com/wshuyi/demo-nlp-classification-fastai.git

Cloning into 'demo-nlp-classification-fastai'...
remote: Enumerating objects: 9, done.
remote: Total 9 (delta 0), reused 0 (delta 0), pack-reused 9
Unpacking objects: 100% (9/9), done.


In [7]:
from fastai import *
from fastai.text import *

**Set the path of the data collections**

In [8]:
path = Path('medical-concept-normalization/data_collection')

In [10]:
# training data set

train_csv = path/'AskAPatient.fold-1.train.csv'
train = pd.read_csv(train_csv, header=None)
#train.head()
#train.info()

In [11]:
# validating data set

valid = pd.read_csv(path/"AskAPatient.fold-1.validation.csv",header=None)
valid.head()

,0,1
0,0,fear
1,2,heels hurt
2,4,Discoloration of skin [blotchy] on the neck
3,6,diastolic dysfunction
4,8,arthritis


In [12]:
# testing data set

test = pd.read_csv(path/"AskAPatient.fold-1.test.csv",header=None)
test.head()

,0,1
0,2,heel pain
1,2,right heel became painful
2,8,Arthritis
3,8,arthristis
4,8,arthritis


In [13]:
# Creating DataBunch for language modelling

data_lm = TextLMDataBunch.from_csv(path, 'cadec_ask.csv')

In [14]:
# Creating DataBunch for classification

# data_clas = TextClasDataBunch.from_df(path, train, valid, test, vocab=data_lm.train_ds.vocab, bs=128)

In [15]:
data_clas = TextClasDataBunch.from_df(path, train, valid, vocab=data_lm.train_ds.vocab, bs=128)

In [16]:
data_clas.add_test(test,label=0)


In [17]:
data_lm.save()
data_clas.save()

In [18]:
data_lm.train_ds.vocab.itos

['xxunk',
 'xxpad',
 'xxbos',
 'xxeos',
 'xxfld',
 'xxmaj',
 'xxup',
 'xxrep',
 'xxwrep',
 'pain',
 'lipitor',
 'in',
 'of',
 'muscle',
 '.',
 'and',
 'to',
 ',',
 'my',
 'i',
 'the',
 'severe',
 'myalgia',
 'fatigue',
 'lower',
 'loss',
 '-',
 'weakness',
 'cramp',
 'limb',
 'stomach',
 'back',
 'a',
 'depression',
 'cramps',
 'leg',
 'gas',
 'joint',
 'memory',
 'upper',
 'legs',
 'pains',
 'it',
 'headache',
 'shoulder',
 'knee',
 'unable',
 '/',
 'neck',
 'not',
 'was',
 'arthralgia',
 'aches',
 'on',
 'foot',
 'excessive',
 'for',
 'like',
 'all',
 'gastrointestinal',
 'hip',
 'numbness',
 'arthritis',
 'insomnia',
 'have',
 'bleeding',
 'walk',
 'heart',
 'feeling',
 'with',
 'problems',
 'arthrotec',
 'nausea',
 'cholesterol',
 'feet',
 'is',
 'tired',
 'asthenia',
 'dizziness',
 'disease',
 'abdominal',
 'walking',
 'this',
 'muscles',
 'difficulty',
 'very',
 'extreme',
 'swelling',
 'chest',
 'hair',
 'burning',
 'that',
 'hand',
 'backache',
 'vision',
 'ache',
 'could',
 's

In [19]:
model_path = path/'models'
model_path.mkdir(exist_ok=True)
url = 'http://files.fast.ai/models/wt103_v1/'
download_url(f'{url}lstm_wt103.pth', model_path/'lstm_wt103.pth')
download_url(f'{url}itos_wt103.pkl', model_path/'itos_wt103.pkl')

In [20]:
learn = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.5)

In [21]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,3.198954,2.706269,0.542438,17:19


In [22]:
# learn.fit_one_cycle(1, 1e-2)

In [23]:
# learn.fit_one_cycle(1, 1e-3)

In [24]:
learn.fit_one_cycle(1, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,2.784176,2.663509,0.546746,17:15


In [25]:
learn.save_encoder('cadec_ask_fold1_first')

In [26]:
learn = text_classifier_learner(data_clas,AWD_LSTM, drop_mult=0.5)
learn.load_encoder('cadec_ask_fold1_first')
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,3.922760,3.857772,0.317191,01:07


In [27]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(5e-3/2., 5e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.901902,3.305874,0.427361,01:17


In [28]:
learn.unfreeze()
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.357673,2.886989,0.490315,02:48


In [29]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.952073,2.546817,0.555690,02:51
1,1.718022,2.390844,0.576271,03:03


In [30]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.541302,2.194413,0.605327,03:04
1,1.319754,2.071553,0.625908,02:59


In [31]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.267479,1.891200,0.645278,03:10
1,1.095509,1.828970,0.663438,03:04


In [32]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.033222,1.758705,0.675545,03:08
1,0.893672,1.675398,0.680387,02:54


In [33]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.842184,1.647524,0.700969,02:52
1,0.740887,1.610365,0.707022,02:52


In [34]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.722731,1.615343,0.713075,03:05
1,0.702222,1.541169,0.728814,02:58
2,0.598792,1.426657,0.742131,03:04
3,0.475397,1.411696,0.742131,02:59


In [35]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.487308,1.535368,0.733656,02:55
1,0.523219,1.468902,0.750605,02:56
2,0.458098,1.404382,0.749395,03:00
3,0.385330,1.416485,0.750605,03:05


In [36]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.388556,1.424601,0.761501,03:02
1,0.404203,1.454774,0.762712,03:00
2,0.382831,1.384104,0.766344,02:57
3,0.302414,1.382846,0.760291,02:59


In [37]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.303685,1.434790,0.767554,03:06
1,0.355793,1.519661,0.766344,03:08
2,0.314995,1.437447,0.773608,03:04
3,0.263104,1.387338,0.772397,03:02


In [38]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.253645,1.517709,0.771186,03:10
1,0.331256,1.478594,0.778450,02:53
2,0.280345,1.466323,0.776029,02:58
3,0.241957,1.402419,0.778450,02:53


In [39]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.231485,1.402295,0.776029,03:00
1,0.256062,1.401371,0.778450,03:03
2,0.245767,1.418444,0.782082,02:52
3,0.196336,1.458507,0.784504,02:54


In [40]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.188218,1.459068,0.789346,02:56
1,0.255857,1.514362,0.790557,02:54
2,0.225640,1.406164,0.795399,02:56
3,0.180258,1.464622,0.800242,02:58


In [41]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.217173,1.489661,0.805085,02:47
1,0.196931,1.384451,0.800242,02:49


In [42]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.190167,1.481656,0.788136,02:51
1,0.195743,1.411085,0.792978,03:04


In [43]:
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.207725,1.477521,0.792978,02:52


In [44]:
# save the best model

learn.save_encoder('cadec_ask_mixed_fold1')

# Part three: Predict on the test dataset

In [45]:
# use the test data for prediction

preds, y = learn.get_preds(DatasetType.Test)

In [46]:
predictions, *_ = learn.get_preds(DatasetType.Test)
labels = np.argmax(predictions, 1)
predict_list = labels.tolist()
standard_list = test[0].tolist()

print(len(predict_list))
print(len(standard_list))

print(predict_list)
print(standard_list)

867
867
[2, 2, 8, 179, 8, 8, 669, 16, 20, 669, 972, 246, 27, 31, 179, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 42, 42, 44, 485, 325, 49, 49, 775, 775, 775, 775, 49, 53, 53, 53, 226, 500, 327, 62, 411, 111, 776, 71, 122, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 289, 80, 714, 80, 80, 80, 80, 491, 689, 88, 89, 233, 92, 92, 92, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 96, 495, 97, 234, 100, 100, 100, 100, 766, 100, 100, 572, 301, 100, 100, 360, 103, 905, 105, 107, 107, 108, 108, 111, 524, 462, 111, 111, 111, 111, 111, 111, 111, 114, 117, 89, 125, 125, 125, 125, 125, 125, 125, 125, 125, 125, 125, 125, 125, 125, 720, 77, 624, 55, 1034, 135, 135, 946, 588, 54, 142, 334, 864, 146, 854, 166, 166, 166, 169, 171, 172, 37, 419, 172, 289, 610, 610, 610, 412, 412, 182, 772, 615, 182, 182, 182, 182, 182, 182, 182, 289, 183, 183, 183, 183, 183, 183, 183, 184, 187, 187, 187, 187, 133, 189, 190, 190,

In [47]:
correct_num = 0

for i in range(0, len(standard_list)):
  if predict_list[i] == standard_list[i]:
    correct_num = correct_num+1
  else:
    pass
print(correct_num)

predict_accuracy = correct_num/len(predict_list)
print(predict_accuracy)

654
0.754325259515571
